# NumPy for Matlab users

### Introduction
MATLAB® and NumPy/SciPy have a lot in common. But there are many differences. NumPy and SciPy were created to do numerical and scientific computing in the most natural way with Python, not to be MATLAB® clones. This page is intended to be a place to collect wisdom about the differences, mostly for the purpose of helping proficient MATLAB® users become proficient NumPy and SciPy users.

### Some Key Differences
| MATLAB| NumPy|
|---|---|
|In MATLAB®, the basic data type is a multidimensional array of double precision floating point numbers. Most expressions take such arrays and return such arrays. Operations on the 2-D instances of these arrays are designed to act more or less like matrix operations in linear algebra.|n NumPy the basic type is a multidimensional array. Operations on these arrays in all dimensionalities including 2D are elementwise operations. However, there is a special matrix type for doing linear algebra, which is just a subclass of the array class. Operations on matrix-class arrays are linear algebra operations.|
|MATLAB® uses 1 (one) based indexing. The initial element of a sequence is found using a(1). See note [INDEXING](https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html#numpy-for-matlab-users-notes)|Python uses 0 (zero) based indexing. The initial element of a sequence is found using a[0].|
|MATLAB®’s scripting language was created for doing linear algebra. The syntax for basic matrix operations is nice and clean, but the API for adding GUIs and making full-fledged applications is more or less an afterthought.| NumPy is based on Python, which was designed from the outset to be an excellent general-purpose programming language. While Matlab’s syntax for some array manipulations is more compact than NumPy’s, NumPy (by virtue of being an add-on to Python) can do many things that Matlab just cannot, for instance subclassing the main array type to do both array and matrix math cleanly.|
|In MATLAB®, arrays have pass-by-value semantics, with a lazy copy-on-write scheme to prevent actually creating copies until they are actually needed. Slice operations copy parts of the array.| In NumPy arrays have pass-by-reference semantics. Slice operations are views into an array.|


### ‘array’ or ‘matrix’? Which should I use?

#### Short answer

#### Use arrays.

They are the standard vector/matrix/tensor type of numpy. Many numpy functions return arrays, not matrices.
There is a clear distinction between element-wise operations and linear algebra operations.
You can have standard vectors or row/column vectors if you like.
The only disadvantage of using the array type is that you will have to use dot instead of * to multiply (reduce) two tensors (scalar product, matrix vector multiplication etc.).



#### Long answer

NumPy contains both an array class and a matrix class. The array class is intended to be a general-purpose n-dimensional array for many kinds of numerical computing, while matrix is intended to facilitate linear algebra computations specifically. In practice there are only a handful of key differences between the two.

* Operator *, dot(), and multiply():
  * For array, ‘``*``’ means element-wise multiplication, and the dot() function is used for matrix multiplication.
  * For matrix, ‘``*``’ means matrix multiplication, and the multiply() function is used for element-wise multiplication.
* Handling of vectors (one-dimensional arrays)
  * For array, the vector shapes 1xN, Nx1, and N are all different things. Operations like A[:,1] return a one-dimensional array of shape N, not a two-dimensional array of shape Nx1. Transpose on a one-dimensional array does nothing.
  * For matrix, one-dimensional arrays are always upconverted to 1xN or Nx1 matrices (row or column vectors). A[:,1] returns a two-dimensional matrix of shape Nx1.
* Handling of higher-dimensional arrays (ndim > 2)
array objects can have number of dimensions > 2;
matrix objects always have exactly two dimensions.
Convenience attributes
  * array has a .T attribute, which returns the transpose of the data.
  * matrix also has .H, .I, and .A attributes, which return the conjugate transpose, inverse, and asarray() of the matrix, respectively.
* Convenience constructor
  * The array constructor takes (nested) Python sequences as initializers. As in, array([[1,2,3],[4,5,6]]).
  * The matrix constructor additionally takes a convenient string initializer. As in matrix("[1 2 3; 4 5 6]").

There are pros and cons to using both:

* array
  * :) You can treat one-dimensional arrays as either row or column vectors. dot(A,v) treats v as a column vector, while dot(v,A) treats v as a row vector. This can save you having to type a lot of transposes.
  * <:( Having to use the dot() function for matrix-multiply is messy – dot(dot(A,B),C) vs. A*B*C.
  * :) Element-wise multiplication is easy: A*B.
  * :) array is the “default” NumPy type, so it gets the most testing, and is the type most likely to be returned by 3rd party code that uses NumPy.
  * :) Is quite at home handling data of any number of dimensions.
  * :) Closer in semantics to tensor algebra, if you are familiar with that.
  * :) All operations (*, /, +, - etc.) are elementwise
* matrix
  * :\\ Behavior is more like that of MATLAB® matrices.
  * <:( Maximum of two-dimensional. To hold three-dimensional data you need array or perhaps a Python list of matrix.
  * <:( Minimum of two-dimensional. You cannot have vectors. They must be cast as single-column or single-row matrices.
  * <:( Since array is the default in NumPy, some functions may return an array even if you give them a matrix as an argument. This shouldn’t happen with NumPy functions (if it does it’s a bug), but 3rd party code based on NumPy may not honor type preservation like NumPy does.
  * :) A*B is matrix multiplication, so more convenient for linear algebra.
  * <:( Element-wise multiplication requires calling a function, multipy(A,B).
  * <:( The use of operator overloading is a bit illogical: * does not work elementwise but / does.

The array is thus much more advisable to use.

### Facilities for Matrix Users
NumPy has some features that facilitate the use of the matrix type, which hopefully make things easier for Matlab converts.

* A matlib module has been added that contains matrix versions of common array constructors like ones(), zeros(), empty(), eye(), rand(), repmat(), etc. Normally these functions return arrays, but the matlib versions return matrix objects.
* mat has been changed to be a synonym for asmatrix, rather than matrix, thus making it a concise way to convert an array to a matrix without copying the data.
* Some top-level functions have been removed. For example numpy.rand() now needs to be accessed as numpy.random.rand(). Or use the rand() from the matlib module. But the “numpythonic” way is to use numpy.random.random(), which takes a tuple for the shape, like other numpy functions.

### Table of Rough MATLAB-NumPy Equivalents
The table below gives rough equivalents for some common MATLAB® expressions. **These are not exact equivalents**, but rather should be taken as hints to get you going in the right direction. For more detail read the built-in documentation on the NumPy functions.

Some care is necessary when writing functions that take arrays or matrices as arguments — if you are expecting an array and are given a matrix, or vice versa, then ‘*’ (multiplication) will give you unexpected results. You can convert back and forth between arrays and matrices using

* asarray: always returns an object of type array
* asmatrix or mat: always return an object of type matrix
* asanyarray: always returns an array object or a subclass derived from it, depending on the input. For instance if you pass in a matrix it returns a matrix.

These functions all accept both arrays and matrices (among other things like Python lists), and thus are useful when writing functions that should accept any array-like object.

In [1]:
import numpy as np
import scipy.linalg as la

###### <span style="color:blue">get the number of dimensions of an array</span>
| MATLAB  |  Numpy |
|---|---|
|ndims(a) |ndim(a) or a.ndim|


In [2]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
m = np.matrix([[1,2,3], [4,5,6]])
print('a=', a)
print(19*'-')
print('m=', m)
print(19*'-')
print('ndim(a) = ', np.ndim(a))
print('a.ndim = ', a.ndim)
print(19*'-')
print('np.ndim(m) = ', np.ndim(m))
print('m.ndim = ', m.ndim)

a= [[1 2 3]
 [4 5 6]
 [7 8 9]]
-------------------
m= [[1 2 3]
 [4 5 6]]
-------------------
ndim(a) =  2
a.ndim =  2
-------------------
np.ndim(m) =  2
m.ndim =  2


###### <span style="color:blue"> get the number of elements of an array</span>

| MATLAB  |  Numpy |
|---|---|
|numel(a) |size(a) or a.size|


In [3]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
m = np.matrix([[1,2,3], [4,5,6]])
print('a=', a)
print(19*'-')
print('m=', m)
print(19*'-')
print('size(a) = ', np.size(a))
print('a.size = ', a.size)
print(19*'-')
print('np.size(m) = ', np.size(m))
print('m.size = ', m.size)

a= [[1 2 3]
 [4 5 6]
 [7 8 9]]
-------------------
m= [[1 2 3]
 [4 5 6]]
-------------------
size(a) =  9
a.size =  9
-------------------
np.size(m) =  6
m.size =  6


###### <span style="color:blue">get the “size” of the matrix</span>
| MATLAB  |  Numpy |
|---|---|
|size(a) |shape(a) or a.shape|

In [4]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
m = np.matrix([[1,2,3], [4,5,6]])
print('a=', a)
print(19*'-')
print('m=', m)
print(19*'-')
print('shape(a) = ', np.shape(a))
print('a.shape = ', a.shape)
print(19*'-')
print('np.shape(m) = ', np.shape(m))
print('m.shape = ', m.shape)

a= [[1 2 3]
 [4 5 6]
 [7 8 9]]
-------------------
m= [[1 2 3]
 [4 5 6]]
-------------------
shape(a) =  (3, 3)
a.shape =  (3, 3)
-------------------
np.shape(m) =  (2, 3)
m.shape =  (2, 3)


###### <span style="color:blue">get the number of elements of the n-th dimension of array a. (Note that MATLAB® uses 1 based indexing while Python uses 0 based indexing, See note [INDEXING](https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html#numpy-for-matlab-users-notes)</span>
| MATLAB  |  Numpy |
|---|---|
|size(a,n) | a.shape[n-1]|


In [5]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
m = np.matrix([[1,2,3], [4,5,6]])
print('a=', a)
print(19*'-')
print('m=', m)
print(19*'-')
print('a.shape[2-1] = ', a.shape[2-1])
print(19*'-')
print('m.shape[2-1] = ', m.shape[2-1])

a= [[1 2 3]
 [4 5 6]
 [7 8 9]]
-------------------
m= [[1 2 3]
 [4 5 6]]
-------------------
a.shape[2-1] =  3
-------------------
m.shape[2-1] =  3


###### <span style="color:blue">2x3 matrix literal</span>
| MATLAB  |  Numpy |
|---|---|
|[ 1 2 3; 4 5 6 ] | array([[1.,2.,3.], [4.,5.,6.]])|

In [6]:
a = np.array([[1.,2.,3.], [4.,5.,6.]])
m = np.matrix([[1.,2.,3.], [4.,5.,6.]])
print('a=', a)
print(19*'-')
print('m=', m)

a= [[ 1.  2.  3.]
 [ 4.  5.  6.]]
-------------------
m= [[ 1.  2.  3.]
 [ 4.  5.  6.]]


###### <span style="color:blue">construct a matrix from blocks a, b, c, and d</span>
| MATLAB  |  Numpy |
|---|---|
|[ a b; c d ]	 | vstack([hstack([a,b]), hstack([c,d])]) or bmat('a b; c d').A|

In [7]:
a = np.array([[1.,2.,3.], [4.,5.,6.]])
b = np.array([[10.,20.,30.], [40.,50.,60.]])
c = np.array([[11.,12.,13.], [14.,15.,16.]])
d = np.array([[19.,29.,39.], [49.,59.,69.]])


m = np.matrix([[1.,2.,3.], [4.,5.,6.]])
n = np.matrix([[10.,20.,30.], [40.,50.,60.]])
r = np.matrix([[11.,12.,13.], [14.,15.,16.]])
p = np.matrix([[19.,29.,39.], [49.,59.,69.]])

In [8]:
e = np.vstack([np.hstack([a,b]), np.hstack([c,d])])
q = np.vstack([np.hstack([m,n]), np.hstack([r,p])])

print ('e = ', e)
print ('q = ', q)

e =  [[  1.   2.   3.  10.  20.  30.]
 [  4.   5.   6.  40.  50.  60.]
 [ 11.  12.  13.  19.  29.  39.]
 [ 14.  15.  16.  49.  59.  69.]]
q =  [[  1.   2.   3.  10.  20.  30.]
 [  4.   5.   6.  40.  50.  60.]
 [ 11.  12.  13.  19.  29.  39.]
 [ 14.  15.  16.  49.  59.  69.]]


In [9]:
f = np.bmat('a b; c d').A
s = np.bmat('m n; r p').A

print ('f = ', f)
print ('s = ', s)

f =  [[  1.   2.   3.  10.  20.  30.]
 [  4.   5.   6.  40.  50.  60.]
 [ 11.  12.  13.  19.  29.  39.]
 [ 14.  15.  16.  49.  59.  69.]]
s =  [[  1.   2.   3.  10.  20.  30.]
 [  4.   5.   6.  40.  50.  60.]
 [ 11.  12.  13.  19.  29.  39.]
 [ 14.  15.  16.  49.  59.  69.]]


###### <span style="color:blue">access last element in the 1xn matrix a</span>
| MATLAB  |  Numpy |
|---|---|
|a(end)	 | a[-1]|

In [10]:
a = np.array([1,2,3,4])
print(a[-1])

4


###### <span style="color:blue">access last element in the 1xn matrix a</span>
| MATLAB  |  Numpy |
|---|---|
|a(2,5)	 | a[1,4]|

In [11]:
a = np.array([[1.,2.,3.,4.,5.], [6.,7.,8.,9.,10.],[11.,12.,13.,14.,15.]])
m = np.matrix([[1.,2.,3.,4.,5.], [6.,7.,8.,9.,10.],[11.,12.,13.,14.,15.]])

print('a=', a)
print(39*'-')
print('m=', m)
print(39*'-')
print(a[1,4])
print(39*'-')
print(m[1,4])

a= [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.]
 [ 11.  12.  13.  14.  15.]]
---------------------------------------
m= [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.]
 [ 11.  12.  13.  14.  15.]]
---------------------------------------
10.0
---------------------------------------
10.0


###### <span style="color:blue">entire second row of a</span>
| MATLAB  |  Numpy |
|---|---|
|a(2,:)| a[1] or a[1,:]|

In [12]:
a = np.array([[1.,2.,3.,4.,5.], [6.,7.,8.,9.,10.],[11.,12.,13.,14.,15.]])
m = np.matrix([[1.,2.,3.,4.,5.], [6.,7.,8.,9.,10.],[11.,12.,13.,14.,15.]])

print('a=', a)
print(39*'-')
print('m=', m)
print(39*'-')
print('2nd row of a: a[1] = ', a[1])
print('2nd row of a: a[1:] = ', a[1,:])
print(39*'-')
print('2nd row of m: m[1] = ', m[1])  
print('2nd row of m: m[1:]', m[1,:])

a= [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.]
 [ 11.  12.  13.  14.  15.]]
---------------------------------------
m= [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.]
 [ 11.  12.  13.  14.  15.]]
---------------------------------------
2nd row of a: a[1] =  [  6.   7.   8.   9.  10.]
2nd row of a: a[1:] =  [  6.   7.   8.   9.  10.]
---------------------------------------
2nd row of m: m[1] =  [[  6.   7.   8.   9.  10.]]
2nd row of m: m[1:] [[  6.   7.   8.   9.  10.]]


###### <span style="color:blue">the first five rows of a</span>
| MATLAB  |  Numpy |
|---|---|
|a(1:5,:)| a[0:5] or a[:5] or a[0:5,:]|

In [13]:
a=np.random.rand(8,5)
print(a)

[[ 0.42889877  0.33620741  0.47738437  0.59888239  0.57585896]
 [ 0.05522298  0.09217045  0.39757494  0.97341008  0.55355556]
 [ 0.88976809  0.04376168  0.47362511  0.7846153   0.57489216]
 [ 0.55314855  0.86649296  0.03492     0.70319555  0.50767356]
 [ 0.57340474  0.84232345  0.58390913  0.38526577  0.89967711]
 [ 0.00653804  0.68068708  0.19925952  0.35977614  0.9561028 ]
 [ 0.09963968  0.78858858  0.49462647  0.88929536  0.91274072]
 [ 0.59365336  0.38232815  0.93805704  0.27750732  0.44905923]]


In [14]:
a[0:5]

array([[ 0.42889877,  0.33620741,  0.47738437,  0.59888239,  0.57585896],
       [ 0.05522298,  0.09217045,  0.39757494,  0.97341008,  0.55355556],
       [ 0.88976809,  0.04376168,  0.47362511,  0.7846153 ,  0.57489216],
       [ 0.55314855,  0.86649296,  0.03492   ,  0.70319555,  0.50767356],
       [ 0.57340474,  0.84232345,  0.58390913,  0.38526577,  0.89967711]])

In [15]:
a[:5]

array([[ 0.42889877,  0.33620741,  0.47738437,  0.59888239,  0.57585896],
       [ 0.05522298,  0.09217045,  0.39757494,  0.97341008,  0.55355556],
       [ 0.88976809,  0.04376168,  0.47362511,  0.7846153 ,  0.57489216],
       [ 0.55314855,  0.86649296,  0.03492   ,  0.70319555,  0.50767356],
       [ 0.57340474,  0.84232345,  0.58390913,  0.38526577,  0.89967711]])

In [16]:
a[0:5,:]

array([[ 0.42889877,  0.33620741,  0.47738437,  0.59888239,  0.57585896],
       [ 0.05522298,  0.09217045,  0.39757494,  0.97341008,  0.55355556],
       [ 0.88976809,  0.04376168,  0.47362511,  0.7846153 ,  0.57489216],
       [ 0.55314855,  0.86649296,  0.03492   ,  0.70319555,  0.50767356],
       [ 0.57340474,  0.84232345,  0.58390913,  0.38526577,  0.89967711]])

In [17]:
m = np.mat(a) #or np.asmatrix(a)

In [18]:
m

matrix([[ 0.42889877,  0.33620741,  0.47738437,  0.59888239,  0.57585896],
        [ 0.05522298,  0.09217045,  0.39757494,  0.97341008,  0.55355556],
        [ 0.88976809,  0.04376168,  0.47362511,  0.7846153 ,  0.57489216],
        [ 0.55314855,  0.86649296,  0.03492   ,  0.70319555,  0.50767356],
        [ 0.57340474,  0.84232345,  0.58390913,  0.38526577,  0.89967711],
        [ 0.00653804,  0.68068708,  0.19925952,  0.35977614,  0.9561028 ],
        [ 0.09963968,  0.78858858,  0.49462647,  0.88929536,  0.91274072],
        [ 0.59365336,  0.38232815,  0.93805704,  0.27750732,  0.44905923]])

In [19]:
m[0:5]

matrix([[ 0.42889877,  0.33620741,  0.47738437,  0.59888239,  0.57585896],
        [ 0.05522298,  0.09217045,  0.39757494,  0.97341008,  0.55355556],
        [ 0.88976809,  0.04376168,  0.47362511,  0.7846153 ,  0.57489216],
        [ 0.55314855,  0.86649296,  0.03492   ,  0.70319555,  0.50767356],
        [ 0.57340474,  0.84232345,  0.58390913,  0.38526577,  0.89967711]])

In [20]:
m[:5]

matrix([[ 0.42889877,  0.33620741,  0.47738437,  0.59888239,  0.57585896],
        [ 0.05522298,  0.09217045,  0.39757494,  0.97341008,  0.55355556],
        [ 0.88976809,  0.04376168,  0.47362511,  0.7846153 ,  0.57489216],
        [ 0.55314855,  0.86649296,  0.03492   ,  0.70319555,  0.50767356],
        [ 0.57340474,  0.84232345,  0.58390913,  0.38526577,  0.89967711]])

In [21]:
m[0:5,:]

matrix([[ 0.42889877,  0.33620741,  0.47738437,  0.59888239,  0.57585896],
        [ 0.05522298,  0.09217045,  0.39757494,  0.97341008,  0.55355556],
        [ 0.88976809,  0.04376168,  0.47362511,  0.7846153 ,  0.57489216],
        [ 0.55314855,  0.86649296,  0.03492   ,  0.70319555,  0.50767356],
        [ 0.57340474,  0.84232345,  0.58390913,  0.38526577,  0.89967711]])

###### <span style="color:blue">the last five rows of a</span>
| MATLAB  |  Numpy |
|---|---|
|a(end-4:end,:)|a[-5:]|

In [22]:
a=np.random.rand(10,5)
a

array([[ 0.89877659,  0.83228871,  0.42230966,  0.82757469,  0.03596778],
       [ 0.65005996,  0.39457308,  0.75818238,  0.28579483,  0.43883847],
       [ 0.9539496 ,  0.55809072,  0.68078805,  0.67504999,  0.0622288 ],
       [ 0.74233815,  0.3939643 ,  0.13201609,  0.21711141,  0.72957886],
       [ 0.86423089,  0.73604093,  0.24398047,  0.02642547,  0.04758567],
       [ 0.35167425,  0.49527549,  0.49673432,  0.42039932,  0.787584  ],
       [ 0.01719551,  0.27119864,  0.57420067,  0.58048633,  0.73362629],
       [ 0.63442791,  0.02092427,  0.45161922,  0.40164021,  0.13517475],
       [ 0.83510998,  0.55772054,  0.46047937,  0.51854881,  0.18769949],
       [ 0.38274974,  0.37951801,  0.70084591,  0.83965991,  0.0314651 ]])

In [23]:
a[-5:]

array([[ 0.35167425,  0.49527549,  0.49673432,  0.42039932,  0.787584  ],
       [ 0.01719551,  0.27119864,  0.57420067,  0.58048633,  0.73362629],
       [ 0.63442791,  0.02092427,  0.45161922,  0.40164021,  0.13517475],
       [ 0.83510998,  0.55772054,  0.46047937,  0.51854881,  0.18769949],
       [ 0.38274974,  0.37951801,  0.70084591,  0.83965991,  0.0314651 ]])

In [24]:
m=np.mat(a)
m

matrix([[ 0.89877659,  0.83228871,  0.42230966,  0.82757469,  0.03596778],
        [ 0.65005996,  0.39457308,  0.75818238,  0.28579483,  0.43883847],
        [ 0.9539496 ,  0.55809072,  0.68078805,  0.67504999,  0.0622288 ],
        [ 0.74233815,  0.3939643 ,  0.13201609,  0.21711141,  0.72957886],
        [ 0.86423089,  0.73604093,  0.24398047,  0.02642547,  0.04758567],
        [ 0.35167425,  0.49527549,  0.49673432,  0.42039932,  0.787584  ],
        [ 0.01719551,  0.27119864,  0.57420067,  0.58048633,  0.73362629],
        [ 0.63442791,  0.02092427,  0.45161922,  0.40164021,  0.13517475],
        [ 0.83510998,  0.55772054,  0.46047937,  0.51854881,  0.18769949],
        [ 0.38274974,  0.37951801,  0.70084591,  0.83965991,  0.0314651 ]])

In [25]:
m[-5:]

matrix([[ 0.35167425,  0.49527549,  0.49673432,  0.42039932,  0.787584  ],
        [ 0.01719551,  0.27119864,  0.57420067,  0.58048633,  0.73362629],
        [ 0.63442791,  0.02092427,  0.45161922,  0.40164021,  0.13517475],
        [ 0.83510998,  0.55772054,  0.46047937,  0.51854881,  0.18769949],
        [ 0.38274974,  0.37951801,  0.70084591,  0.83965991,  0.0314651 ]])

###### <span style="color:blue">rows one to three and columns five to nine of a. This gives read-only access.</span>
| MATLAB  |  Numpy |
|---|---|
|a(1:3,5:9)	|a[0:3][:,4:9]|

In [26]:
a=np.random.rand(5,10)
a

array([[ 0.41675838,  0.56876949,  0.45929584,  0.00640037,  0.47490682,
         0.81158433,  0.35510969,  0.44315677,  0.13986964,  0.82510417],
       [ 0.69099261,  0.22525244,  0.22671291,  0.07612265,  0.62169353,
         0.08135902,  0.37983178,  0.53350436,  0.59691156,  0.3561862 ],
       [ 0.14177781,  0.02767032,  0.34565463,  0.89574374,  0.52362848,
         0.02913406,  0.93820831,  0.37324484,  0.66774609,  0.12819359],
       [ 0.77689477,  0.62938186,  0.79703513,  0.01609144,  0.0869374 ,
         0.0065842 ,  0.1594474 ,  0.5746946 ,  0.52622777,  0.1925812 ],
       [ 0.80156176,  0.02412992,  0.50830696,  0.84278004,  0.22337921,
         0.43970784,  0.82122358,  0.11724688,  0.32987948,  0.6857091 ]])

In [27]:
a[0:3][:,4:9]

array([[ 0.47490682,  0.81158433,  0.35510969,  0.44315677,  0.13986964],
       [ 0.62169353,  0.08135902,  0.37983178,  0.53350436,  0.59691156],
       [ 0.52362848,  0.02913406,  0.93820831,  0.37324484,  0.66774609]])

###### <span style="color:blue">rows 2,4 and 5 and columns 1 and 3. This allows the matrix to be modified, and doesn’t require a regular slice</span>
| MATLAB  |  Numpy |
|---|---|
|a([2,4,5],[1,3])	|a[np.ix\_([1,3,4],[0,2])]|

In [28]:
a[np.ix_([1,3,4],[0,2])]

array([[ 0.69099261,  0.22671291],
       [ 0.77689477,  0.79703513],
       [ 0.80156176,  0.50830696]])

###### <span style="color:blue">every other row of a, starting with the third and going to the twenty-first</span>
| MATLAB  |  Numpy |
|---|---|
|a(3:2:21,:)|a[ 2:21:2,:]|

In [29]:
a[2:21:2,:]

array([[ 0.14177781,  0.02767032,  0.34565463,  0.89574374,  0.52362848,
         0.02913406,  0.93820831,  0.37324484,  0.66774609,  0.12819359],
       [ 0.80156176,  0.02412992,  0.50830696,  0.84278004,  0.22337921,
         0.43970784,  0.82122358,  0.11724688,  0.32987948,  0.6857091 ]])

###### <span style="color:blue">every other row of a, starting with the first</span>
| MATLAB  |  Numpy |
|---|---|
|a(1:2:end,:)|a[ ::2,:]|

In [30]:
a[::2,:]

array([[ 0.41675838,  0.56876949,  0.45929584,  0.00640037,  0.47490682,
         0.81158433,  0.35510969,  0.44315677,  0.13986964,  0.82510417],
       [ 0.14177781,  0.02767032,  0.34565463,  0.89574374,  0.52362848,
         0.02913406,  0.93820831,  0.37324484,  0.66774609,  0.12819359],
       [ 0.80156176,  0.02412992,  0.50830696,  0.84278004,  0.22337921,
         0.43970784,  0.82122358,  0.11724688,  0.32987948,  0.6857091 ]])

###### <span style="color:blue">a with rows in reverse order</span>
| MATLAB  |  Numpy |
|---|---|
|a(end:-1:1,:) or flipud(a)|a[ ::-1,:]|

In [31]:
a[ ::-1,:]

array([[ 0.80156176,  0.02412992,  0.50830696,  0.84278004,  0.22337921,
         0.43970784,  0.82122358,  0.11724688,  0.32987948,  0.6857091 ],
       [ 0.77689477,  0.62938186,  0.79703513,  0.01609144,  0.0869374 ,
         0.0065842 ,  0.1594474 ,  0.5746946 ,  0.52622777,  0.1925812 ],
       [ 0.14177781,  0.02767032,  0.34565463,  0.89574374,  0.52362848,
         0.02913406,  0.93820831,  0.37324484,  0.66774609,  0.12819359],
       [ 0.69099261,  0.22525244,  0.22671291,  0.07612265,  0.62169353,
         0.08135902,  0.37983178,  0.53350436,  0.59691156,  0.3561862 ],
       [ 0.41675838,  0.56876949,  0.45929584,  0.00640037,  0.47490682,
         0.81158433,  0.35510969,  0.44315677,  0.13986964,  0.82510417]])

###### <span style="color:blue">a with copy of the first row appended to the end</span>
| MATLAB  |  Numpy |
|---|---|
|a([1:end 1],:)	|a[np.r\_[:len(a),0]]	|

In [32]:
a[np.r_[:len(a),0]]

array([[ 0.41675838,  0.56876949,  0.45929584,  0.00640037,  0.47490682,
         0.81158433,  0.35510969,  0.44315677,  0.13986964,  0.82510417],
       [ 0.69099261,  0.22525244,  0.22671291,  0.07612265,  0.62169353,
         0.08135902,  0.37983178,  0.53350436,  0.59691156,  0.3561862 ],
       [ 0.14177781,  0.02767032,  0.34565463,  0.89574374,  0.52362848,
         0.02913406,  0.93820831,  0.37324484,  0.66774609,  0.12819359],
       [ 0.77689477,  0.62938186,  0.79703513,  0.01609144,  0.0869374 ,
         0.0065842 ,  0.1594474 ,  0.5746946 ,  0.52622777,  0.1925812 ],
       [ 0.80156176,  0.02412992,  0.50830696,  0.84278004,  0.22337921,
         0.43970784,  0.82122358,  0.11724688,  0.32987948,  0.6857091 ],
       [ 0.41675838,  0.56876949,  0.45929584,  0.00640037,  0.47490682,
         0.81158433,  0.35510969,  0.44315677,  0.13986964,  0.82510417]])

###### <span style="color:blue">transpose of a</span>
| MATLAB  |  Numpy |
|---|---|
|a.'|a.transpose() or a.T|

In [33]:
a.transpose()

array([[ 0.41675838,  0.69099261,  0.14177781,  0.77689477,  0.80156176],
       [ 0.56876949,  0.22525244,  0.02767032,  0.62938186,  0.02412992],
       [ 0.45929584,  0.22671291,  0.34565463,  0.79703513,  0.50830696],
       [ 0.00640037,  0.07612265,  0.89574374,  0.01609144,  0.84278004],
       [ 0.47490682,  0.62169353,  0.52362848,  0.0869374 ,  0.22337921],
       [ 0.81158433,  0.08135902,  0.02913406,  0.0065842 ,  0.43970784],
       [ 0.35510969,  0.37983178,  0.93820831,  0.1594474 ,  0.82122358],
       [ 0.44315677,  0.53350436,  0.37324484,  0.5746946 ,  0.11724688],
       [ 0.13986964,  0.59691156,  0.66774609,  0.52622777,  0.32987948],
       [ 0.82510417,  0.3561862 ,  0.12819359,  0.1925812 ,  0.6857091 ]])

In [34]:
a.T

array([[ 0.41675838,  0.69099261,  0.14177781,  0.77689477,  0.80156176],
       [ 0.56876949,  0.22525244,  0.02767032,  0.62938186,  0.02412992],
       [ 0.45929584,  0.22671291,  0.34565463,  0.79703513,  0.50830696],
       [ 0.00640037,  0.07612265,  0.89574374,  0.01609144,  0.84278004],
       [ 0.47490682,  0.62169353,  0.52362848,  0.0869374 ,  0.22337921],
       [ 0.81158433,  0.08135902,  0.02913406,  0.0065842 ,  0.43970784],
       [ 0.35510969,  0.37983178,  0.93820831,  0.1594474 ,  0.82122358],
       [ 0.44315677,  0.53350436,  0.37324484,  0.5746946 ,  0.11724688],
       [ 0.13986964,  0.59691156,  0.66774609,  0.52622777,  0.32987948],
       [ 0.82510417,  0.3561862 ,  0.12819359,  0.1925812 ,  0.6857091 ]])

###### <span style="color:blue">aconjugate transpose of a</span>
| MATLAB  |  Numpy |
|---|---|
|a'|a.conj().transpose() or a.conj().T|

In [35]:
a.conj().T

array([[ 0.41675838,  0.69099261,  0.14177781,  0.77689477,  0.80156176],
       [ 0.56876949,  0.22525244,  0.02767032,  0.62938186,  0.02412992],
       [ 0.45929584,  0.22671291,  0.34565463,  0.79703513,  0.50830696],
       [ 0.00640037,  0.07612265,  0.89574374,  0.01609144,  0.84278004],
       [ 0.47490682,  0.62169353,  0.52362848,  0.0869374 ,  0.22337921],
       [ 0.81158433,  0.08135902,  0.02913406,  0.0065842 ,  0.43970784],
       [ 0.35510969,  0.37983178,  0.93820831,  0.1594474 ,  0.82122358],
       [ 0.44315677,  0.53350436,  0.37324484,  0.5746946 ,  0.11724688],
       [ 0.13986964,  0.59691156,  0.66774609,  0.52622777,  0.32987948],
       [ 0.82510417,  0.3561862 ,  0.12819359,  0.1925812 ,  0.6857091 ]])

In [36]:
a.conj().transpose()

array([[ 0.41675838,  0.69099261,  0.14177781,  0.77689477,  0.80156176],
       [ 0.56876949,  0.22525244,  0.02767032,  0.62938186,  0.02412992],
       [ 0.45929584,  0.22671291,  0.34565463,  0.79703513,  0.50830696],
       [ 0.00640037,  0.07612265,  0.89574374,  0.01609144,  0.84278004],
       [ 0.47490682,  0.62169353,  0.52362848,  0.0869374 ,  0.22337921],
       [ 0.81158433,  0.08135902,  0.02913406,  0.0065842 ,  0.43970784],
       [ 0.35510969,  0.37983178,  0.93820831,  0.1594474 ,  0.82122358],
       [ 0.44315677,  0.53350436,  0.37324484,  0.5746946 ,  0.11724688],
       [ 0.13986964,  0.59691156,  0.66774609,  0.52622777,  0.32987948],
       [ 0.82510417,  0.3561862 ,  0.12819359,  0.1925812 ,  0.6857091 ]])

###### <span style="color:blue">matrix multiply</span>
| MATLAB  |  Numpy |
|---|---|
|a * b	|a.dot(b)|

In [37]:
a.dot(np.asarray(m))

array([[ 2.60291799,  2.07837145,  2.48649025,  2.32208848,  1.33226526],
       [ 2.58597407,  1.90064887,  1.80262062,  1.89979245,  0.76102755],
       [ 2.46251478,  1.75787529,  1.68126205,  1.72756035,  1.60824711],
       [ 2.83762942,  1.83518878,  2.10287905,  2.12736506,  0.67429253],
       [ 2.82076964,  2.34393877,  2.2440073 ,  2.65773514,  1.74467602]])

###### <span style="color:blue">element-wise multiply</span>
| MATLAB  |  Numpy |
|---|---|
|a .* b	|a * b|

In [38]:
a*a

array([[  1.73687550e-01,   3.23498730e-01,   2.10952670e-01,
          4.09647483e-05,   2.25536491e-01,   6.58669129e-01,
          1.26102895e-01,   1.96387925e-01,   1.95635165e-02,
          6.80796897e-01],
       [  4.77470794e-01,   5.07386634e-02,   5.13987433e-02,
          5.79465819e-03,   3.86502843e-01,   6.61929072e-03,
          1.44272181e-01,   2.84626900e-01,   3.56303413e-01,
          1.26868607e-01],
       [  2.01009467e-02,   7.65646554e-04,   1.19477122e-01,
          8.02356840e-01,   2.74186788e-01,   8.48793333e-04,
          8.80234838e-01,   1.39311708e-01,   4.45884845e-01,
          1.64335964e-02],
       [  6.03565477e-01,   3.96121528e-01,   6.35264992e-01,
          2.58934472e-04,   7.55811181e-03,   4.33517204e-05,
          2.54234729e-02,   3.30273885e-01,   2.76915661e-01,
          3.70875178e-02],
       [  6.42501259e-01,   5.82253035e-04,   2.58375963e-01,
          7.10278201e-01,   4.98982716e-02,   1.93342985e-01,
          6.74408175e-01

###### <span style="color:blue">element-wise divide</span>
| MATLAB  |  Numpy |
|---|---|
|a./ b	|a/b|

In [39]:
a/a

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]])

###### <span style="color:blue">element-wise exponentiation</span>
| MATLAB  |  Numpy |
|---|---|
|a.^3		|a\*\*3	|

In [40]:
a**3

array([[  7.23857427e-02,   1.83996207e-01,   9.68896841e-02,
          2.62189585e-07,   1.07108819e-01,   5.34565545e-01,
          4.47803605e-02,   8.70306388e-02,   2.73634204e-03,
          5.61728361e-01],
       [  3.29928792e-01,   1.14290079e-02,   1.16527586e-02,
          4.41104750e-04,   2.40286316e-01,   5.38539030e-04,
          5.47991591e-02,   1.51849692e-01,   2.12681627e-01,
          4.51888466e-02],
       [  2.84986816e-03,   2.11856844e-05,   4.12978200e-02,
          7.18706113e-01,   1.43572012e-01,   2.47287941e-05,
          8.25843642e-01,   5.19973757e-02,   2.97737863e-01,
          2.10668171e-03],
       [  4.68906860e-01,   2.49311705e-01,   5.06328513e-01,
          4.16662876e-06,   6.57082602e-04,   2.85436498e-07,
          4.05370661e-03,   1.89806619e-01,   1.45720709e-01,
          7.14235861e-03],
       [  5.15004442e-01,   1.40497191e-05,   1.31334300e-01,
          5.98608293e-01,   1.11462365e-02,   8.50144266e-02,
          5.53839898e-01

###### <span style="color:blue">matrix whose i,jth element is (a_ij > 0.5). The Matlab result is an array of 0s and 1s. The NumPy result is an array of the boolean values False and True.</span>
| MATLAB  |  Numpy |
|---|---|
|(a>0.5)	|(a>0.5)|

In [41]:
a>0.5

array([[False,  True, False, False, False,  True, False, False, False,
         True],
       [ True, False, False, False,  True, False, False,  True,  True,
        False],
       [False, False, False,  True,  True, False,  True, False,  True,
        False],
       [ True,  True,  True, False, False, False, False,  True,  True,
        False],
       [ True, False,  True,  True, False, False,  True, False, False,
         True]], dtype=bool)

###### <span style="color:blue">find the indices where (a > 0.5)</span>
| MATLAB  |  Numpy |
|---|---|
|find(a>0.5)|nonzero(a>0.5)|

In [42]:
np.nonzero(a>0.5)

(array([0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4]),
 array([1, 5, 9, 0, 4, 7, 8, 3, 4, 6, 8, 0, 1, 2, 7, 8, 0, 2, 3, 6, 9]))

###### <span style="color:blue">extract the columms of a where vector v > 0.5</span>
| MATLAB  |  Numpy |
|---|---|
|a(:,find(v>0.5))|a[:,nonzero(v>0.5)[0]]|

In [43]:
v = a[:,4].T
a[:,np.nonzero(v>0.5)[0]]

array([[ 0.56876949,  0.45929584],
       [ 0.22525244,  0.22671291],
       [ 0.02767032,  0.34565463],
       [ 0.62938186,  0.79703513],
       [ 0.02412992,  0.50830696]])

###### <span style="color:blue">extract the columms of a where column vector v > 0.5</span>
| MATLAB  |  Numpy |
|---|---|
|a(:,find(v>0.5))|a[:,v.T>0.5]	|

In [44]:
a[:,v.T>0.5]

/home/jimmy/.local/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 10 but corresponding boolean dimension is 5
  if __name__ == '__main__':


array([[ 0.56876949,  0.45929584],
       [ 0.22525244,  0.22671291],
       [ 0.02767032,  0.34565463],
       [ 0.62938186,  0.79703513],
       [ 0.02412992,  0.50830696]])

###### <span style="color:blue">a with elements less than 0.5 zeroed out</span>
| MATLAB  |  Numpy |
|---|---|
|a(a<0.5)=0	|a[a<0.5]=0|

In [45]:
a[a<0.5]=0
a

array([[ 0.        ,  0.56876949,  0.        ,  0.        ,  0.        ,
         0.81158433,  0.        ,  0.        ,  0.        ,  0.82510417],
       [ 0.69099261,  0.        ,  0.        ,  0.        ,  0.62169353,
         0.        ,  0.        ,  0.53350436,  0.59691156,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.89574374,  0.52362848,
         0.        ,  0.93820831,  0.        ,  0.66774609,  0.        ],
       [ 0.77689477,  0.62938186,  0.79703513,  0.        ,  0.        ,
         0.        ,  0.        ,  0.5746946 ,  0.52622777,  0.        ],
       [ 0.80156176,  0.        ,  0.50830696,  0.84278004,  0.        ,
         0.        ,  0.82122358,  0.        ,  0.        ,  0.6857091 ]])